<a href="https://colab.research.google.com/github/run-llama/llama_index/blob/main/docs/docs/examples/low_level/fusion_retriever.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="在 Colab 中打开"/></a>


# 从零开始构建高级融合检索器

在本教程中，我们将向您展示如何从零开始构建一个高级检索器。

具体来说，我们将向您展示如何从头开始构建我们的`QueryFusionRetriever`。

这在很大程度上受到了RAG-fusion仓库的启发，网址为：https://github.com/Raudaschl/rag-fusion。


## 设置

我们加载文档并构建一个简单的向量索引。


In [ ]:
%pip install llama-index-readers-file pymupdf
%pip install llama-index-llms-openai
%pip install llama-index-retrievers-bm25

In [ ]:
import nest_asyncio

nest_asyncio.apply()

#### 加载文档


In [ ]:
!mkdir data
!wget --user-agent "Mozilla" "https://arxiv.org/pdf/2307.09288.pdf" -O "data/llama2.pdf"

--2024-04-03 09:32:31--  https://arxiv.org/pdf/2307.09288.pdf
Resolving arxiv.org (arxiv.org)... 151.101.3.42, 151.101.131.42, 151.101.67.42, ...
Connecting to arxiv.org (arxiv.org)|151.101.3.42|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13661300 (13M) [application/pdf]
Saving to: ‘data/llama2.pdf’

data/llama2.pdf     100%[===================>]  13.03M  7.44MB/s    in 1.8s    

2024-04-03 09:32:33 (7.44 MB/s) - ‘data/llama2.pdf’ saved [13661300/13661300]



如果您在Colab上打开此笔记本，您可能需要安装LlamaIndex 🦙。


In [ ]:
!pip install llama-index

In [ ]:
from pathlib import Path
from llama_index.readers.file import PyMuPDFReader

loader = PyMuPDFReader()
documents = loader.load(file_path="./data/llama2.pdf")

```python
# 设置模型
```

这里是设置模型的部分。


In [ ]:
import os

os.environ["OPENAI_API_KEY"] = "sk-..."

In [ ]:
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding

llm = OpenAI(model="gpt-3.5-turbo", temperature=0.1)
embed_model = OpenAIEmbedding(
    model="text-embedding-3-small", embed_batch_size=256
)

#### 加载到向量存储中


In [ ]:
from llama_index.core import VectorStoreIndex
from llama_index.core.node_parser import SentenceSplitter

splitter = SentenceSplitter(chunk_size=1024)
index = VectorStoreIndex.from_documents(
    documents, transformations=[splitter], embed_model=embed_model
)

## 定义高级检索器

我们定义一个高级检索器，执行以下步骤：
1. 查询生成/重写：根据原始用户查询生成多个查询。
2. 对每个查询在一组检索器上执行检索。
3. 重新排序/融合：融合所有查询的结果，并对“融合”出的前几个相关结果应用重新排序步骤！

然后在下一节中，我们将把这个模块插入到我们的响应合成模块中。


### 第一步：查询生成/重写

第一步是从原始查询中生成查询，以更好地匹配查询意图，并提高检索结果的精确度/召回率。例如，我们可以将查询重写为更小的查询。

我们可以通过提示ChatGPT来实现这一点。


In [ ]:
from llama_index.core import PromptTemplate

In [ ]:
query_str = "How do the models developed in this work compare to open-source chat models based on the benchmarks tested?"

In [ ]:
query_gen_prompt_str = (
    "You are a helpful assistant that generates multiple search queries based on a "
    "single input query. Generate {num_queries} search queries, one on each line, "
    "related to the following input query:\n"
    "Query: {query}\n"
    "Queries:\n"
)
query_gen_prompt = PromptTemplate(query_gen_prompt_str)

In [ ]:
def generate_queries(llm, query_str: str, num_queries: int = 4):
    fmt_prompt = query_gen_prompt.format(
        num_queries=num_queries - 1, query=query_str
    )
    response = llm.complete(fmt_prompt)
    queries = response.text.split("\n")
    return queries

In [ ]:
queries = generate_queries(llm, query_str, num_queries=4)

In [ ]:
print(queries)

['1. Comparison of models developed in this work to open-source chat models in benchmark testing', '2. Performance evaluation of models developed in this work versus open-source chat models on tested benchmarks', '3. Analysis of differences between models developed in this work and open-source chat models in benchmark assessments']


### 步骤2：对每个查询执行向量搜索

现在我们对每个查询运行检索。这意味着我们从每个向量存储中获取前k个最相关的结果。

**注意**：我们也可以有多个检索器。那么我们运行的总查询数量为N*M，其中N是检索器的数量，M是生成的查询数量。因此将会有N*M个检索列表。

在这里，我们将使用从我们的向量存储中提供的检索器。如果您想了解如何从头开始构建这个，请参阅[我们的教程](https://docs.llamaindex.ai/en/latest/examples/low_level/retrieval.html#put-this-into-a-retriever)。


In [ ]:
from tqdm.asyncio import tqdmasync def run_queries(queries, retrievers):    """对检索器运行查询。"""    tasks = []    for query in queries:        for i, retriever in enumerate(retrievers):            tasks.append(retriever.aretrieve(query))    task_results = await tqdm.gather(*tasks)    results_dict = {}    for i, (query, query_result) in enumerate(zip(queries, task_results)):        results_dict[(query, i)] = query_result    return results_dict

In [ ]:
# 获取检索器from llama_index.retrievers.bm25 import BM25Retriever## 向量检索器vector_retriever = index.as_retriever(similarity_top_k=2)## bm25检索器bm25_retriever = BM25Retriever.from_defaults(    docstore=index.docstore, similarity_top_k=2)

In [ ]:
results_dict = await run_queries(queries, [vector_retriever, bm25_retriever])

  0%|          | 0/6 [00:00<?, ?it/s]

100%|██████████| 6/6 [00:00<00:00, 11.14it/s]


### 步骤 3：执行融合

这里的下一步是执行融合：将多个检索器的结果合并为一个，并重新排名。

请注意，给定节点可能会从不同的检索器中多次检索到，因此需要一种方法来去重和重新排名给定多次检索的节点。

我们将向您展示如何执行“倒数排名融合”：对于每个节点，将其在每个检索列表中的倒数排名相加。

然后按得分从高到低重新排序节点。

完整论文链接：https://plg.uwaterloo.ca/~gvcormac/cormacksigir09-rrf.pdf


In [ ]:
from typing import Listfrom llama_index.core.schema import NodeWithScoredef fuse_results(results_dict, similarity_top_k: int = 2):    """融合结果。"""    k = 60.0  # `k`是一个用于控制异常值排名影响的参数。    fused_scores = {}    text_to_node = {}    # 计算倒数排名分数    for nodes_with_scores in results_dict.values():        for rank, node_with_score in enumerate(            sorted(                nodes_with_scores, key=lambda x: x.score or 0.0, reverse=True            )        ):            text = node_with_score.node.get_content()            text_to_node[text] = node_with_score            if text not in fused_scores:                fused_scores[text] = 0.0            fused_scores[text] += 1.0 / (rank + k)    # 排序结果    reranked_results = dict(        sorted(fused_scores.items(), key=lambda x: x[1], reverse=True)    )    # 调整节点分数    reranked_nodes: List[NodeWithScore] = []    for text, score in reranked_results.items():        reranked_nodes.append(text_to_node[text])        reranked_nodes[-1].score = score    return reranked_nodes[:similarity_top_k]

In [ ]:
final_results = fuse_results(results_dict)

In [ ]:
for n in final_results:
    print(n.score, "\n", n.text, "\n********\n")

0.03333333333333333 
 Figure 12: Human evaluation results for Llama 2-Chat models compared to open- and closed-source models
across ~4,000 helpfulness prompts with three raters per prompt.
The largest Llama 2-Chat model is competitive with ChatGPT. Llama 2-Chat 70B model has a win rate of
36% and a tie rate of 31.5% relative to ChatGPT. Llama 2-Chat 70B model outperforms PaLM-bison chat
model by a large percentage on our prompt set. More results and analysis is available in Section A.3.7.
Inter-Rater Reliability (IRR).
In our human evaluations, three different annotators provided independent
assessments for each model generation comparison. High IRR scores (closer to 1.0) are typically seen as
better from a data quality perspective, however, context is important. Highly subjective tasks like evaluating
the overall helpfulness of LLM generations will usually have lower IRR scores than more objective labelling
tasks. There are relatively few public benchmarks for these contexts, so we fe

**分析**：上面的代码有一些简单的组件。
1. 遍历每个检索列表中的节点，并将其倒数排名添加到节点的ID中。节点的ID是其文本的哈希值，用于去重。
2. 按照从高分到低分的顺序对结果进行排序。
3. 调整节点得分。


## 插入到RetrieverQueryEngine

现在我们已经准备好将其定义为一个自定义的检索器，并将其插入到我们的`RetrieverQueryEngine`中（负责检索和综合）。


In [ ]:
from typing import Listfrom llama_index.core import QueryBundlefrom llama_index.core.retrievers import BaseRetrieverfrom llama_index.core.schema import NodeWithScoreimport asyncioclass FusionRetriever(BaseRetriever):    """带有融合功能的集成检索器。"""    def __init__(        self,        llm,        retrievers: List[BaseRetriever],        similarity_top_k: int = 2,    ) -> None:        """初始化参数。"""        self._retrievers = retrievers        self._similarity_top_k = similarity_top_k        self._llm = llm        super().__init__()    def _retrieve(self, query_bundle: QueryBundle) -> List[NodeWithScore]:        """检索。"""        queries = generate_queries(            self._llm, query_bundle.query_str, num_queries=4        )        results = asyncio.run(run_queries(queries, self._retrievers))        final_results = fuse_results(            results, similarity_top_k=self._similarity_top_k        )        return final_results

In [ ]:
from llama_index.core.query_engine import RetrieverQueryEngine

fusion_retriever = FusionRetriever(
    llm, [vector_retriever, bm25_retriever], similarity_top_k=2
)

query_engine = RetrieverQueryEngine(fusion_retriever)

In [ ]:
response = query_engine.query(query_str)

In [ ]:
print(str(response))

The models developed in this work, specifically the Llama 2-Chat models, outperform open-source chat models on most benchmarks that were tested.
